In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#!ls "/content/gdrive/MyDrive/"

In [3]:
import torch
from torch.utils.data import DataLoader
import torch.utils.data as Data
from torchvision import datasets, transforms
import numpy as np
import os
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

class Model_Drop(nn.Module):
    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 600)
        self.fc2 = nn.Linear(600, 128)
        self.fc3 = nn.Linear(128, 10)
        self.drop_layer = nn.Dropout(p=0.25)

    def last_hidden_layer_output(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.drop_layer(F.relu(self.conv3(x)))
        x = self.drop_layer(F.relu(self.conv4(x)))
        x = x.view(-1, 7*7*64)
        x = self.drop_layer(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x


In [6]:
correct_ratio = []
alpha = 0.05
beta = 1 - alpha

In [7]:
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transforms.ToTensor())

trainload = DataLoader(train_data, batch_size=128, shuffle=True)
testload = DataLoader(test_data, batch_size=128, shuffle=False)



Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
teach_model = Model_Drop()
teach_model.to(device)
teach_model.load_state_dict(torch.load("/content/gdrive/MyDrive/TENET_FASHION_MNIST_model_cnn_teacher_T_100.pt"))
teach_model.eval()

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
)

In [9]:
model = Model_Drop()
model.to(device)

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
)

In [10]:
criterion = nn.CrossEntropyLoss()
#criterion2 = nn.KLDivLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

torch.manual_seed(2)
torch.cuda.manual_seed(2)

In [11]:
model.train()
T = 100

def dist_loss(t, s, temp):
    prob_t = F.softmax(t/temp, dim=1)
    log_prob_s = F.log_softmax(s/temp, dim=1)
    dist_loss = -(prob_t*log_prob_s).sum(dim=1).mean()
    return dist_loss

for epoch in range(50):

    print("Current epoch is : ", epoch)

    model.train()
    loss_sigma = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainload):

        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs.float())

        teacher_outputs = teach_model(inputs.float())

        #loss1 = F.cross_entropy(outputs, labels)

        loss = dist_loss(teacher_outputs, outputs, T)

        #loss = alpha * loss1 + beta * dist_loss(teacher_outputs, outputs, T)

        loss.backward()

        optimizer.step()




Current epoch is :  0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Current epoch is :  1
Current epoch is :  2
Current epoch is :  3
Current epoch is :  4
Current epoch is :  5
Current epoch is :  6
Current epoch is :  7
Current epoch is :  8
Current epoch is :  9
Current epoch is :  10
Current epoch is :  11
Current epoch is :  12
Current epoch is :  13
Current epoch is :  14
Current epoch is :  15
Current epoch is :  16
Current epoch is :  17
Current epoch is :  18
Current epoch is :  19
Current epoch is :  20
Current epoch is :  21
Current epoch is :  22
Current epoch is :  23
Current epoch is :  24
Current epoch is :  25
Current epoch is :  26
Current epoch is :  27
Current epoch is :  28
Current epoch is :  29
Current epoch is :  30
Current epoch is :  31
Current epoch is :  32
Current epoch is :  33
Current epoch is :  34
Current epoch is :  35
Current epoch is :  36
Current epoch is :  37
Current epoch is :  38
Current epoch is :  39
Current epoch is :  40
Current epoch is :  41
Current epoch is :  42
Current epoch is :  43
Current epoch is :  

In [12]:
total_loss, total_err = 0., 0.

model.eval()

for X, y in testload:
    X, y = X.to(device), y.to(device)
    
    outputs = model(X.float())

    total_err += (outputs.max(dim=1)[1] != y).sum().item()

print("total test error is ", total_err / len(testload.dataset))

total test error is  0.0738


In [13]:
model.eval()
torch.save(model.state_dict(), "TENET_FASHION_MNIST_model_cnn_student_T_100.pt")


In [14]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/TENET_FASHION_MNIST_model_cnn_student_T_100.pt")